In [1]:
import requests
import pandas as pd 
import json
import numpy as py 
from flask import Flask, render_template, redirect
from splinter import Browser
from bs4 import BeautifulSoup as bs
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager
from sqlalchemy import create_engine,inspect


In [2]:
player_url = "https://www.balldontlie.io/api/v1/players"
player_api = requests.get(player_url).json()

In [3]:
url_query2 = "https://www.balldontlie.io/api/v1/teams"
teams_api = requests.get(url_query2).json()

In [4]:
url_query3 = "https://www.balldontlie.io/api/v1/games"
games_api = requests.get(url_query3).json()

In [5]:
url_query4 = "https://www.balldontlie.io/api/v1/stats"
stats_api = requests.get(url_query4).json()

In [6]:
# ****we need to give parameters to Run this API not sure how we want to use this API****
# url_query5 = "https://www.balldontlie.io/api/v1/season_averages"
url_query5 = "https://www.balldontlie.io/api/v1/season_averages?season=2018&player_ids[]=1&player_ids[]=2"
season_avg_api = requests.get(url_query5).json()

In [7]:
season_avg_api

{'data': [{'games_played': 31,
   'player_id': 1,
   'season': 2018,
   'min': '18:50',
   'fgm': 1.74,
   'fga': 5.0,
   'fg3m': 1.26,
   'fg3a': 4.03,
   'ftm': 0.39,
   'fta': 0.42,
   'oreb': 0.16,
   'dreb': 1.39,
   'reb': 1.55,
   'ast': 0.65,
   'stl': 0.55,
   'blk': 0.19,
   'turnover': 0.45,
   'pf': 1.68,
   'pts': 5.13,
   'fg_pct': 0.348,
   'fg3_pct': 0.312,
   'ft_pct': 0.923},
  {'games_played': 34,
   'player_id': 2,
   'season': 2018,
   'min': '12:34',
   'fgm': 1.12,
   'fga': 3.24,
   'fg3m': 0.74,
   'fg3a': 2.18,
   'ftm': 0.21,
   'fta': 0.26,
   'oreb': 0.32,
   'dreb': 1.47,
   'reb': 1.79,
   'ast': 1.94,
   'stl': 0.38,
   'blk': 0.15,
   'turnover': 0.82,
   'pf': 1.32,
   'pts': 3.18,
   'fg_pct': 0.345,
   'fg3_pct': 0.338,
   'ft_pct': 0.778}]}

In [8]:
player_api


{'data': [{'id': 14,
   'first_name': 'Ike',
   'height_feet': None,
   'height_inches': None,
   'last_name': 'Anigbogu',
   'position': 'C',
   'team': {'id': 12,
    'abbreviation': 'IND',
    'city': 'Indiana',
    'conference': 'East',
    'division': 'Central',
    'full_name': 'Indiana Pacers',
    'name': 'Pacers'},
   'weight_pounds': None},
  {'id': 25,
   'first_name': 'Ron',
   'height_feet': None,
   'height_inches': None,
   'last_name': 'Baker',
   'position': 'G',
   'team': {'id': 20,
    'abbreviation': 'NYK',
    'city': 'New York',
    'conference': 'East',
    'division': 'Atlantic',
    'full_name': 'New York Knicks',
    'name': 'Knicks'},
   'weight_pounds': None},
  {'id': 47,
   'first_name': 'Jabari',
   'height_feet': None,
   'height_inches': None,
   'last_name': 'Bird',
   'position': 'G',
   'team': {'id': 2,
    'abbreviation': 'BOS',
    'city': 'Boston',
    'conference': 'East',
    'division': 'Atlantic',
    'full_name': 'Boston Celtics',
    'name

In [9]:
player_df = pd.json_normalize(player_api, record_path =['data'])
player_df

,id,first_name,height_feet,height_inches,last_name,position,weight_pounds,team.id,team.abbreviation,team.city,team.conference,team.division,team.full_name,team.name
0,14,Ike,NaN,NaN,Anigbogu,C,NaN,12,IND,Indiana,East,Central,Indiana Pacers,Pacers
1,25,Ron,NaN,NaN,Baker,G,NaN,20,NYK,New York,East,Atlantic,New York Knicks,Knicks
2,47,Jabari,NaN,NaN,Bird,G,NaN,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics
3,67,MarShon,NaN,NaN,Brooks,G,NaN,15,MEM,Memphis,West,Southwest,Memphis Grizzlies,Grizzlies
4,71,Lorenzo,NaN,NaN,Brown,G,NaN,28,TOR,Toronto,East,Atlantic,Toronto Raptors,Raptors
5,90,Omri,NaN,NaN,Casspi,F,NaN,15,MEM,Memphis,West,Southwest,Memphis Grizzlies,Grizzlies
6,1,Alex,6.0,6.0,Abrines,G,200.0,21,OKC,Oklahoma City,West,Northwest,Oklahoma City Thunder,Thunder
7,119,Tyler,NaN,NaN,Davis,C,NaN,21,OKC,Oklahoma City,West,Northwest,Oklahoma City Thunder,Thunder
8,149,Keenan,NaN,NaN,Evans,G,NaN,9,DET,Detroit,East,Central,Detroit Pistons,Pistons
9,179,Marcin,NaN,NaN,Gortat,C,NaN,13,LAC,LA,West,Pacific,LA Clippers,Clippers


In [10]:
player_df.drop(['weight_pounds','team.abbreviation','team.city','team.conference','team.division','team.name'], axis=1, inplace=True)


In [11]:
player_df

,id,first_name,height_feet,height_inches,last_name,position,team.id,team.full_name
0,14,Ike,NaN,NaN,Anigbogu,C,12,Indiana Pacers
1,25,Ron,NaN,NaN,Baker,G,20,New York Knicks
2,47,Jabari,NaN,NaN,Bird,G,2,Boston Celtics
3,67,MarShon,NaN,NaN,Brooks,G,15,Memphis Grizzlies
4,71,Lorenzo,NaN,NaN,Brown,G,28,Toronto Raptors
5,90,Omri,NaN,NaN,Casspi,F,15,Memphis Grizzlies
6,1,Alex,6.0,6.0,Abrines,G,21,Oklahoma City Thunder
7,119,Tyler,NaN,NaN,Davis,C,21,Oklahoma City Thunder
8,149,Keenan,NaN,NaN,Evans,G,9,Detroit Pistons
9,179,Marcin,NaN,NaN,Gortat,C,13,LA Clippers


In [12]:
player_df.drop(['height_feet', 'height_inches'], axis=1, inplace=True)

In [13]:
player_df

,id,first_name,last_name,position,team.id,team.full_name
0,14,Ike,Anigbogu,C,12,Indiana Pacers
1,25,Ron,Baker,G,20,New York Knicks
2,47,Jabari,Bird,G,2,Boston Celtics
3,67,MarShon,Brooks,G,15,Memphis Grizzlies
4,71,Lorenzo,Brown,G,28,Toronto Raptors
5,90,Omri,Casspi,F,15,Memphis Grizzlies
6,1,Alex,Abrines,G,21,Oklahoma City Thunder
7,119,Tyler,Davis,C,21,Oklahoma City Thunder
8,149,Keenan,Evans,G,9,Detroit Pistons
9,179,Marcin,Gortat,C,13,LA Clippers


In [14]:
player_final_df = player_df
player_final_df

,id,first_name,last_name,position,team.id,team.full_name
0,14,Ike,Anigbogu,C,12,Indiana Pacers
1,25,Ron,Baker,G,20,New York Knicks
2,47,Jabari,Bird,G,2,Boston Celtics
3,67,MarShon,Brooks,G,15,Memphis Grizzlies
4,71,Lorenzo,Brown,G,28,Toronto Raptors
5,90,Omri,Casspi,F,15,Memphis Grizzlies
6,1,Alex,Abrines,G,21,Oklahoma City Thunder
7,119,Tyler,Davis,C,21,Oklahoma City Thunder
8,149,Keenan,Evans,G,9,Detroit Pistons
9,179,Marcin,Gortat,C,13,LA Clippers


In [15]:
player_final_df2 = player_final_df.rename(columns={'id': 'player_id', 'position': 'positions', 'team.id':'team_id', 'team.full_name':'team_full_name'})
player_final_df2

,player_id,first_name,last_name,positions,team_id,team_full_name
0,14,Ike,Anigbogu,C,12,Indiana Pacers
1,25,Ron,Baker,G,20,New York Knicks
2,47,Jabari,Bird,G,2,Boston Celtics
3,67,MarShon,Brooks,G,15,Memphis Grizzlies
4,71,Lorenzo,Brown,G,28,Toronto Raptors
5,90,Omri,Casspi,F,15,Memphis Grizzlies
6,1,Alex,Abrines,G,21,Oklahoma City Thunder
7,119,Tyler,Davis,C,21,Oklahoma City Thunder
8,149,Keenan,Evans,G,9,Detroit Pistons
9,179,Marcin,Gortat,C,13,LA Clippers


In [16]:
teams_df = pd.json_normalize(teams_api, record_path =['data'])
teams_df

,id,abbreviation,city,conference,division,full_name,name
0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
1,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics
2,3,BKN,Brooklyn,East,Atlantic,Brooklyn Nets,Nets
3,4,CHA,Charlotte,East,Southeast,Charlotte Hornets,Hornets
4,5,CHI,Chicago,East,Central,Chicago Bulls,Bulls
5,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers
6,7,DAL,Dallas,West,Southwest,Dallas Mavericks,Mavericks
7,8,DEN,Denver,West,Northwest,Denver Nuggets,Nuggets
8,9,DET,Detroit,East,Central,Detroit Pistons,Pistons
9,10,GSW,Golden State,West,Pacific,Golden State Warriors,Warriors


In [17]:
teams_df.drop(['abbreviation','city','conference','division','name'], axis=1, inplace=True)

In [18]:
teams_df


,id,full_name
0,1,Atlanta Hawks
1,2,Boston Celtics
2,3,Brooklyn Nets
3,4,Charlotte Hornets
4,5,Chicago Bulls
5,6,Cleveland Cavaliers
6,7,Dallas Mavericks
7,8,Denver Nuggets
8,9,Detroit Pistons
9,10,Golden State Warriors


In [19]:
teams_final_df = teams_df.rename(columns={'id': 'team_id', 'full_name':'team_full_name'})
teams_final_df

,team_id,team_full_name
0,1,Atlanta Hawks
1,2,Boston Celtics
2,3,Brooklyn Nets
3,4,Charlotte Hornets
4,5,Chicago Bulls
5,6,Cleveland Cavaliers
6,7,Dallas Mavericks
7,8,Denver Nuggets
8,9,Detroit Pistons
9,10,Golden State Warriors


In [20]:
games_df = pd.json_normalize(games_api, record_path =['data'])
games_df

,id,date,home_team_score,period,postseason,season,status,time,visitor_team_score,home_team.id,...,home_team.division,home_team.full_name,home_team.name,visitor_team.id,visitor_team.abbreviation,visitor_team.city,visitor_team.conference,visitor_team.division,visitor_team.full_name,visitor_team.name
0,47179,2019-01-30T00:00:00.000Z,126,4,False,2018,Final,,94,2,...,Atlantic,Boston Celtics,Celtics,4,CHA,Charlotte,East,Southeast,Charlotte Hornets,Hornets
1,48751,2019-02-09T00:00:00.000Z,112,4,False,2018,Final,,123,2,...,Atlantic,Boston Celtics,Celtics,13,LAC,LA,West,Pacific,LA Clippers,Clippers
2,48739,2019-02-08T00:00:00.000Z,117,4,False,2018,Final,,110,23,...,Atlantic,Philadelphia 76ers,76ers,8,DEN,Denver,West,Northwest,Denver Nuggets,Nuggets
3,48740,2019-02-08T00:00:00.000Z,119,4,False,2018,Final,,106,30,...,Southeast,Washington Wizards,Wizards,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers
4,48746,2019-02-08T00:00:00.000Z,102,4,False,2018,Final,,96,26,...,Pacific,Sacramento Kings,Kings,16,MIA,Miami,East,Southeast,Miami Heat,Heat
5,48762,2019-02-11T00:00:00.000Z,121,4,False,2018,Final,,112,9,...,Central,Detroit Pistons,Pistons,30,WAS,Washington,East,Southeast,Washington Wizards,Wizards
6,48745,2019-02-08T00:00:00.000Z,122,4,False,2018,Final,,117,19,...,Southwest,New Orleans Pelicans,Pelicans,18,MIN,Minnesota,West,Northwest,Minnesota Timberwolves,Timberwolves
7,48748,2019-02-09T00:00:00.000Z,105,4,False,2018,Final,,90,12,...,Central,Indiana Pacers,Pacers,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers
8,48749,2019-02-09T00:00:00.000Z,120,4,False,2018,Final,,129,1,...,Southeast,Atlanta Hawks,Hawks,4,CHA,Charlotte,East,Southeast,Charlotte Hornets,Hornets
9,48747,2019-02-09T00:00:00.000Z,125,4,False,2018,Final,,105,29,...,Northwest,Utah Jazz,Jazz,27,SAS,San Antonio,West,Southwest,San Antonio Spurs,Spurs


In [21]:
stats_df = pd.json_normalize(stats_api, record_path =['data'])
stats_df

,id,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,...,player.position,player.team_id,player.weight_pounds,team.id,team.abbreviation,team.city,team.conference,team.division,team.full_name,team.name
0,1069008,0.0,1.0,2.0,0.200,5.0,1.0,0.333,9.0,3.0,...,G,3,215.0,26,SAC,Sacramento,West,Pacific,Sacramento Kings,Kings
1,1069009,4.0,0.0,5.0,0.000,2.0,0.0,0.200,5.0,1.0,...,F,10,234.0,26,SAC,Sacramento,West,Pacific,Sacramento Kings,Kings
2,1069010,4.0,1.0,5.0,0.000,0.0,0.0,0.667,6.0,4.0,...,C,7,240.0,26,SAC,Sacramento,West,Pacific,Sacramento Kings,Kings
3,1069011,1.0,0.0,1.0,0.545,11.0,6.0,0.500,18.0,9.0,...,G,26,214.0,26,SAC,Sacramento,West,Pacific,Sacramento Kings,Kings
4,1069012,8.0,1.0,5.0,0.000,2.0,0.0,0.400,10.0,4.0,...,G,26,175.0,26,SAC,Sacramento,West,Pacific,Sacramento Kings,Kings
5,1069013,2.0,0.0,1.0,0.500,2.0,1.0,0.615,13.0,8.0,...,G,1,205.0,26,SAC,Sacramento,West,Pacific,Sacramento Kings,Kings
6,1069014,2.0,0.0,1.0,0.333,3.0,1.0,0.400,5.0,2.0,...,G,13,180.0,26,SAC,Sacramento,West,Pacific,Sacramento Kings,Kings
7,1069015,1.0,0.0,4.0,0.500,2.0,1.0,0.545,11.0,6.0,...,F,26,234.0,26,SAC,Sacramento,West,Pacific,Sacramento Kings,Kings
8,1069016,2.0,0.0,3.0,0.250,4.0,1.0,0.400,5.0,2.0,...,F,24,210.0,26,SAC,Sacramento,West,Pacific,Sacramento Kings,Kings
9,1069017,4.0,0.0,4.0,0.000,0.0,0.0,0.000,3.0,0.0,...,F-C,13,240.0,26,SAC,Sacramento,West,Pacific,Sacramento Kings,Kings


In [23]:
connection_string = "postgres:Dontforget123!@localhost:5432/Basketball_stats"
engine = create_engine(f'postgresql://{connection_string}')

In [25]:
# Confirm tables
engine.table_names()

/var/folders/mc/nvjx38yn6j14pqksc9yx1pk40000gn/T/ipykernel_39235/3015872754.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['players', 'teams']

In [25]:
inspector = inspect(engine)
table_names = inspector.get_table_names()

In [26]:
table_names

['players', 'teams']

In [27]:
player_final_df2.to_sql(name='players', con=engine, if_exists='replace', index=True)

In [28]:
teams_final_df.to_sql(name='teams', con=engine, if_exists='replace', index=True)